In [32]:
window = tk.Tk()
frame_a=tk.Frame()
frame_b=tk.Frame()
greeting=tk.Label(text="Hello, Tkinter", master = frame_a)
label=tk.Label(text="Hello, Tkinter", width=10,height=10, master = frame_b)
entry=tk.Text(fg="yellow",bg="blue",width=50, master = frame_b)
button=tk.Button(text="Click me!",width=25,height=5,bg="blue",fg="yellow", master = frame_a)
frame_a.pack(side=tk.LEFT)
frame_b.pack(side=tk.RIGHT)
greeting.pack()
label.pack()
entry.pack(side=tk.LEFT)
button.pack(side=tk.RIGHT)
window.mainloop()

KeyboardInterrupt: 

In [2]:
import tkinter as tk
app = tk.Tk()
app.rowconfigure(0, minsize=800, weight=1)
app.columnconfigure(1, minsize=800, weight=1)
txt_edit = tk.Text(app)
result = tk.Text()
frm_buttons = tk.Frame(app, relief=tk.RAISED, bd=2)
btn_open = tk.Button(frm_buttons, text="Open")
btn_save = tk.Button(frm_buttons, text="Save As...")

btn_open.grid(row=0, column=0, sticky="ew", padx=5, pady=5)
btn_save.grid(row=1, column=0, sticky="ew", padx=5)
result.grid(row = 0, column = 2)
frm_buttons.grid(row=0, column=0, sticky="ns")
txt_edit.grid(row=0, column=1, sticky="nsew")

app.mainloop()

In [75]:
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [2]:
btn_color = 'blue'

In [90]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def encode_categorical(dataframe, categorical_columns):
    print(categorical_columns)
    encoder = OneHotEncoder(sparse=False)
    for col in categorical_columns:
            encoded = encoder.fit_transform(dataframe[[col]])
            encoded_df = pd.DataFrame(encoded, columns=[f"{col}_{category}" for category in encoder.categories_[0]])
            dataframe = pd.concat([dataframe, encoded_df], axis=1).drop(col, axis=1)
    print(dataframe)
    return dataframe

def compute_and_visualize_correlation(dataframe, target_column):
    correlation_matrix = dataframe.corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')
    plt.show()
    return correlation_matrix[target_column].sort_values(ascending=False)

def identify_high_correlation(dataframe, threshold=0.8):
    correlation_matrix = dataframe.corr()
    high_correlation = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                colname = correlation_matrix.columns[i]
                high_correlation.add(colname)

    return high_correlation

def remove_outliers_using_percentiles(data, continious_columns, lower_percentile=0.25, upper_percentile=0.95, multiplier=1.5):
    numeric_data = data[continious_columns]
    Q1 = numeric_data.quantile(lower_percentile)
    Q3 = numeric_data.quantile(upper_percentile)
    IQR = Q3 - Q1
    filtered_data = data[~((numeric_data < (Q1 - multiplier * IQR)) | (numeric_data > (Q3 + multiplier * IQR))).any(axis=1)]
    return filtered_data

def get_columns_info(data):
    column_types = data.columns.to_series().groupby(data.dtypes).groups
    categorial = [column for column in data.columns if len(data[column].unique()) < 10]
    continious_columns = [column for key in column_types for column in column_types[key] if key!="object" and column not in categorial]
    print(categorial)
    print(continious_columns)
    return (categorial, continious_columns)

def remove_na(data):
    old_size = len(data)
    cleaned_data = data.dropna()
    new_size = len(cleaned_data)

    if old_size!=new_size:
        print(f'There were {old_size - new_size} records removed')
    else:
        print("No missing data")
    return cleaned_data

def process_columns(data):
    cleaned_data = remove_na(data)
    categorical_cols, continuous_cols = get_columns_info(cleaned_data)
    cleaned_data = remove_outliers_using_percentiles(cleaned_data, continuous_cols)
    cleaned_data = cleaned_data[categorical_cols+continuous_cols]
    total_cols = len(categorical_cols) + len(continuous_cols)
    cols_per_row = 3 
    rows = math.ceil(total_cols / cols_per_row)
    fig, axes = plt.subplots(rows, cols_per_row, figsize=(15, 5 * rows))
    axes = axes.flatten()
    for idx, col in enumerate(categorical_cols):
        proportion = cleaned_data[col].value_counts(normalize=True)
        sns.barplot(x=proportion.index, y=proportion.values, ax=axes[idx])
        axes[idx].set_title(f'Proportions for {col}')
        axes[idx].set_ylabel('Proportion')
        axes[idx].set_xlabel(col)
    for idx, col in enumerate(continuous_cols, start=len(categorical_cols)):
        sns.kdeplot(cleaned_data[col], fill=True, ax=axes[idx])
        axes[idx].set_title(f'Density for {col}')
        axes[idx].set_ylabel('Density')
        axes[idx].set_xlabel(col)
    for ax in axes[len(categorical_cols) + len(continuous_cols):]:
        ax.axis('off')

In [95]:
def get_text():
    name = df_name.get()
    df_name2.insert(0, name)
    with zipfile.ZipFile(name, 'r') as z:
        with z.open('train.csv') as f:
            data = pd.read_csv(f)
    df_head.insert("0.0", str(data.head()))

    cleaned_data = remove_na(data)
    categorical_cols, continuous_cols = get_columns_info(cleaned_data)
    cleaned_data = remove_outliers_using_percentiles(cleaned_data, continuous_cols)
    cleaned_data = cleaned_data[categorical_cols+continuous_cols]
    total_cols = len(categorical_cols) + len(continuous_cols)
    cols_per_row = 3 
    rows = math.ceil(total_cols / cols_per_row)
    fig, axes = plt.subplots(rows, cols_per_row, figsize=(15, 5 * rows))
    axes = axes.flatten()
    for idx, col in enumerate(categorical_cols):
        proportion = cleaned_data[col].value_counts(normalize=True)
        sns.barplot(x=proportion.index, y=proportion.values, ax=axes[idx])
        axes[idx].set_title(f'Proportions for {col}')
        axes[idx].set_ylabel('Proportion')
        axes[idx].set_xlabel(col)
    for idx, col in enumerate(continuous_cols, start=len(categorical_cols)):
        sns.kdeplot(cleaned_data[col], fill=True, ax=axes[idx])
        axes[idx].set_title(f'Density for {col}')
        axes[idx].set_ylabel('Density')
        axes[idx].set_xlabel(col)
    for ax in axes[len(categorical_cols) + len(continuous_cols):]:
        ax.axis('off')
    canvas.figure = fig
    canvas.draw()

    

In [96]:
import tkinter as tk 

user_interface = tk.Tk()

user_interface.rowconfigure(0, minsize=10, weight=1)
user_interface.columnconfigure(1, minsize=10, weight=1)

df_frame = tk.Frame(master = user_interface)
df_frame2 = tk.Frame(master = user_interface)

df_frame.grid(row = 0, column = 0)
df_frame2.grid(row = 0, column = 1)

btn_read_csv = tk.Button(master = df_frame, text = "Read File", bg=btn_color, command = get_text)
fig, ax = plt.subplots()
canvas = FigureCanvasTkAgg(fig, master = df_frame2)

df_name = tk.Entry(df_frame)
df_name2 = tk.Entry(df_frame)
lbl_name = tk.Label(df_frame, text = "Dataset Name")
df_head = tk.Text(df_frame2)

# df_frame2.grid(row = 1, column = 1)

btn_read_csv.pack()
df_name.pack()
df_name2.pack()
df_head.pack()
canvas.get_tk_widget().pack()

# lbl_name.grid(row = 1, column = 1)

user_interface.mainloop()


There were 29 records removed
['Gender', 'MaritalStatus', 'DependentChildren', 'DependentsOther', 'PartTimeFullTime', 'DaysWorkedPerWeek']
['Age', 'InitialIncurredCalimsCost', 'WeeklyWages', 'HoursWorkedPerWeek', 'UltimateIncurredClaimCost']


In [54]:
import tkinter as tk

window = tk.Tk()

for i in range(3):
    for j in range(3):
        frame = tk.Frame(
            master=window,
            relief=tk.RAISED,
            borderwidth=1
        )
        frame.grid(row=i, column=j)
        label = tk.Label(master=frame, text=f"Row {i}\nColumn {j}")
        label.pack()

window.mainloop()

In [19]:
test = tk.Tk()

label = tk.Label(text = "My Text", bg = "blue", width=10, height=10)
label2 = tk.Label(text = "My Text2")
entry = tk.Entry()
label.grid(row = 0, column = 0)
label2.grid(row = 1, column = 1)
entry.grid(row = 2, column = 2)
test.mainloop()

KeyboardInterrupt: 

In [20]:
entry.get()

'5323'